# Open AI Assistant QuickStart
Based on YouTube tutorial on OpenAI Assistants [here](https://www.youtube.com/watch?v=5rcjGjgJNQc)

In [2]:
from openai import OpenAI
import os
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

In [3]:
assistant = client.beta.assistants.create(
    name="Math tutor", 
    instructions="You are a math tutor.  Write and run code to answer math problems.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-1106-preview" 
)
assistant

Assistant(id='asst_Hb5uRyRt5BcgGafy3K6l0uF7', created_at=1727883841, description=None, instructions='You are a math tutor.  Write and run code to answer math problems.', metadata={}, model='gpt-4-1106-preview', name='Math tutor', object='assistant', tools=[CodeInterpreterTool(type='code_interpreter')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=[]), file_search=None), top_p=1.0)

In [4]:
thread = client.beta.threads.create()
thread

Thread(id='thread_KtlYXMY3R6A2M4EyFKz6d1D0', created_at=1727883845, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [5]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role = "user",
    content = "Solve this problem. 3x + 11 = 14"
    #content = "Solve this problem. What are the values of x in the following equation. 6x + 11 = 23"
    #content = "What are the values of x in the following equation\n\n\ 6x^2 - 11x - 10 = 0"
)
message

Message(id='msg_tKgrvlapgUqM1axYKW11TZdr', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Solve this problem. 3x + 11 = 14'), type='text')], created_at=1727883850, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_KtlYXMY3R6A2M4EyFKz6d1D0')

In [6]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id, 
    instructions="Please start every response with 'Well then'"
)

In [11]:
# Only continue once the status has changed from "queued" to "complete" or something similar
run.status

'queued'

In [12]:
run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
)
run


Run(id='run_wZDfSb6Jgl2Zp20wAJHEJ8GM', assistant_id='asst_Hb5uRyRt5BcgGafy3K6l0uF7', cancelled_at=None, completed_at=1727883870, created_at=1727883859, expires_at=None, failed_at=None, incomplete_details=None, instructions="Please start every response with 'Well then'", last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, response_format='auto', started_at=1727883860, status='completed', thread_id='thread_KtlYXMY3R6A2M4EyFKz6d1D0', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=Usage(completion_tokens=154, prompt_tokens=598, total_tokens=752), temperature=1.0, top_p=1.0, tool_resources={}, parallel_tool_calls=True)

In [13]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)
messages

SyncCursorPage[Message](data=[Message(id='msg_XqnsI728YjcL2IXKokGqmjvz', assistant_id='asst_Hb5uRyRt5BcgGafy3K6l0uF7', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Well then, the solution to the equation \\( 3x + 11 = 14 \\) is \\( x = 1 \\).'), type='text')], created_at=1727883870, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_wZDfSb6Jgl2Zp20wAJHEJ8GM', status=None, thread_id='thread_KtlYXMY3R6A2M4EyFKz6d1D0'), Message(id='msg_bvvDZk4tXUdw5Fx5pmFn1p9U', assistant_id='asst_Hb5uRyRt5BcgGafy3K6l0uF7', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Well then, to solve for \\( x \\) in the equation \\( 3x + 11 = 14 \\), we need to isolate \\( x \\). First, we subtract 11 from both sides of the equation, and then we divide by 3. Let me calculate that for you.'), type='text')], created_at=1727883861, incomplete_at=None, incomplete_

In [14]:
for message in reversed(messages.data):
    print(message.role, message.content[0].text.value)

user Solve this problem. 3x + 11 = 14
assistant Well then, to solve for \( x \) in the equation \( 3x + 11 = 14 \), we need to isolate \( x \). First, we subtract 11 from both sides of the equation, and then we divide by 3. Let me calculate that for you.
assistant Well then, the solution to the equation \( 3x + 11 = 14 \) is \( x = 1 \).


Link to files with knowledge retrieval


In [16]:
file = client.files.create(
    purpose="assistants",
    file=open("resources/strictly-student-guide.pdf", "rb")
)
file

FileObject(id='file-nbo4s8iAt5VETHu2v5pXfXoQ', bytes=64381, created_at=1727884011, filename='strictly-student-guide.pdf', object='file', purpose='assistants', status='processed', status_details=None)

In [17]:
doc_assistant = client.beta.assistants.create(
    name="Strictly tutor", 
    instructions="Please read the document and answer the questions.",
    tools=[{"type": "file_search"}],
    model="gpt-4-1106-preview" 
    #file_ids=[file.id]
)
doc_assistant

Assistant(id='asst_IIAszbbo0jb04fBtTcq8rtkF', created_at=1727884045, description=None, instructions='Please read the document and answer the questions.', metadata={}, model='gpt-4-1106-preview', name='Strictly tutor', object='assistant', tools=[FileSearchTool(type='file_search', file_search={'ranking_options': {'ranker': 'default_2024_08_21', 'score_threshold': 0.0}})], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=[])), top_p=1.0)

In [18]:
doc_thread = client.beta.threads.create()
doc_thread

Thread(id='thread_wy3ehb5rP6TxuRrGiT56tlmY', created_at=1727884051, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [19]:
message = client.beta.threads.messages.create(
    thread_id=doc_thread.id,
    role = "user",
    
    #file_id = file.id,
    #content = "Name the Strictly judges."
    content = "Describe the Strictly TV series."
    #content = "Solve this problem. What are the values of x in the following equation. 6x + 11 = 23"
    #content = "What are the values of x in the following equation\n\n\ 6x^2 - 11x - 10 = 0"
)
message

Message(id='msg_9NjuXoMpvuVtJSCE664SaaXk', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Describe the Strictly TV series.'), type='text')], created_at=1727884062, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_wy3ehb5rP6TxuRrGiT56tlmY')

In [20]:
doc_run = client.beta.threads.runs.create(
    thread_id=doc_thread.id,
    assistant_id=doc_assistant.id
)

In [22]:
doc_run.status

'queued'

In [23]:
doc_run = client.beta.threads.runs.retrieve(
    thread_id=doc_thread.id,
    run_id=doc_run.id
)

In [ ]:
doc_run.status

In [ ]:
messages = client.beta.threads.messages.list(
    thread_id=doc_thread.id
)
messages

In [ ]:
for message in reversed(messages.data):
    print(message.role, message.content[0].text.value)